In [1]:
#Imports
# type: ignore
import os
from dotenv import load_dotenv
import pandas as pd

from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import DataFrameLoader
from langchain.docstore.document import Document

In [2]:
# Import des variables d'environnement (.env)

load_dotenv()

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
azure_deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

In [3]:
# Initialisation du modèle d'Embeddings

embedding_model = AzureOpenAIEmbeddings(
    openai_api_key=azure_openai_api_key,
    deployment="text-embedding-3-large",
    azure_endpoint=azure_openai_api_endpoint,
    openai_api_version="2023-05-15",
    chunk_size=500
)

/tmp/ipykernel_1083/1634952500.py:3: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(


In [5]:
# Import du dataset 

gutenberg_data = pd.read_csv("Data/gutenberg_ebooks.csv")

In [6]:
gutenberg_data.tail()

,Ebook ID,Author,Title,Credits,Summary,Language,LoC Class,Subject,Subject_2,Subject_3,Subject_4,Category,EBook-No.,Release Date,Most Recently Updated,Copyright Status,Downloads
19924,19996,Various,"The Atlantic Monthly, Volume 16, No. 96, Octob...","Produced by Joshua Hutchinson, Josephine Paolu...","""The Atlantic Monthly, Volume 16, No. 96, Octo...",English,AP: General Works: Periodicals,American periodicals,NaN,NaN,NaN,Text,19996,"Dec 2, 2006",NaN,Public domain in the USA.,88 downloads in the last 30 days.
19925,19997,"Ober, Frederick A. (Frederick Albion), 1849-1913",Amerigo Vespucci,"Produced by Suzanne Shell, Richard J. Shiffer ...","""Amerigo Vespucci"" by Frederick A. Ober is a h...",English,E011: History: America: America,"Vespucci, Amerigo, 1451-1512",NaN,NaN,NaN,Text,19997,"Dec 3, 2006",NaN,Public domain in the USA.,145 downloads in the last 30 days.
19926,19998,"Allen, Lewis Falley, 1800-1890",Rural ArchitectureBeing a Complete Description...,"Produced by Louise Hope, Steven Giacomelli and...","""Rural Architecture"" by Lewis Falley Allen is ...",English,NA: Fine Arts: Architecture,Domestic animals,"Architecture, Domestic",NaN,NaN,Text,19998,"Dec 3, 2006",NaN,Public domain in the USA.,232 downloads in the last 30 days.
19927,19999,"Trowbridge, J. T. (John Townsend), 1827-1916",The Drummer Boy,Produced by David Edwards and the Online Distr...,"""The Drummer Boy"" by J. T. Trowbridge is a his...",English,PZ: Language and Literatures: Juvenile belles ...,"United States -- History -- Civil War, 1861-18...",NaN,NaN,NaN,Text,19999,"Dec 3, 2006",NaN,Public domain in the USA.,76 downloads in the last 30 days.
19928,20000,"Verne, Jules, 1828-1905",Twenty Thousand Leagues Under the Sea,NaN,NaN,English,PQ: Language and Literatures: Romance literatu...,Science fiction,Submarines (Ships) -- Fiction,Sea stories,Underwater exploration -- Fiction,Sound,20000,"Sep 1, 2007","Jan 30, 2021",Public domain in the USA.,515 downloads in the last 30 days.


In [6]:
gutenberg_data.shape

(19929, 17)

In [7]:
print(gutenberg_data.isnull().sum())

Ebook ID                     0
Author                     874
Title                        0
Credits                   1563
Summary                    819
Language                     0
LoC Class                    8
Subject                      9
Subject_2                 8253
Subject_3                13595
Subject_4                16257
Category                     0
EBook-No.                    0
Release Date                 0
Most Recently Updated     1821
Copyright Status             0
Downloads                    0
dtype: int64


In [7]:
gutenberg_data = gutenberg_data.dropna(subset=["Summary", "Title", "Author"])

In [8]:
gutenberg_data.shape

(18335, 17)

In [9]:
print(gutenberg_data.isnull().sum())

Ebook ID                     0
Author                       0
Title                        0
Credits                    758
Summary                      0
Language                     0
LoC Class                    2
Subject                      3
Subject_2                 7558
Subject_3                12487
Subject_4                14962
Category                     0
EBook-No.                    0
Release Date                 0
Most Recently Updated     1731
Copyright Status             0
Downloads                    0
dtype: int64


In [10]:
metadata_columns = ["Title", "Author", "Language", "Release Date", "Subject", "Subject_2", "Subject_3", "Subject_4"]

gutenberg_data[metadata_columns] = gutenberg_data[metadata_columns].fillna('')


In [11]:
# Fichier .Faiss trop lourd pour être commité. Random pour ne garder que environ 8000 ref et pouvoir commit
gutenberg_data = gutenberg_data.sample(n=8250, random_state=42)

In [12]:
gutenberg_data.shape

(8250, 17)

In [13]:
# type: ignore
from langchain.schema import Document

documents = []

for idx, row in gutenberg_data.iterrows():

    # On va combiner l'ensemble des sujets en un seul bloc
    subjects = ', '.join([row[col] for col in ["Subject", "Subject_2", "Subject_3", "Subject_4"] if row[col]])
    
    # Création du dictionnaire des métadonnées avec rassemblement de toutes les informations clé
    metadata = {
        "Title": row["Title"],
        "Author": row["Author"],
        "Language": row["Language"],
        "Release Date": row["Release Date"],
        "Subjects": subjects
    }
    
    document = Document(page_content=row["Summary"], metadata=metadata)
    documents.append(document)

In [15]:
# type: ignore

# Utilisation avec FAISS (= moteur de stockage vectoriel)
# Objectif : Permettre une recherche de similarité efficace

# Le kernel crash lors de la création de l'index vectoriel :
# TEST 1 : Toujours avec FAISS mais avec un traitement par lots (batch)
# TEST 2 SI PERSISTENCE ERREURS AVEC FAIS : Utiliser Chroma

# Onn passe de 39 lots à 17 après suppression des livres pour tenter d'avoir un fichier avec une taille correcte

from langchain.vectorstores import FAISS

# Mise en place du traitement par lot : 
batch_size = 500

vectorstore = None

"""index = VectorstoreIndexCreator(
    embedding=embedding_model,
    vectorstore_cls=FAISS

    # Création de l'index vectoriel
).from_documents(documents)"""

# Traitement des documents par lots
for i in range(0, len(documents), batch_size):
    batch_docs = documents[i:i+batch_size]
    print(f"Traitement du lot {i//batch_size + 1}/{(len(documents) + batch_size - 1)//batch_size}")

    if vectorstore is None:
        vectorstore = FAISS.from_documents(batch_docs, embedding_model)
    else:
        new_vectorstore = FAISS.from_documents(batch_docs, embedding_model)
        vectorstore.merge_from(new_vectorstore)

# Enregistrement de l'index complet
vectorstore.save_local("book_index")

Traitement du lot 1/17
Traitement du lot 2/17
Traitement du lot 3/17
Traitement du lot 4/17
Traitement du lot 5/17
Traitement du lot 6/17
Traitement du lot 7/17
Traitement du lot 8/17
Traitement du lot 9/17
Traitement du lot 10/17
Traitement du lot 11/17
Traitement du lot 12/17
Traitement du lot 13/17
Traitement du lot 14/17
Traitement du lot 15/17
Traitement du lot 16/17
Traitement du lot 17/17


In [14]:
print(f"Nombre de docs créés : {len(documents)}")

Nombre de docs créés : 8250


In [15]:
# Import du fichier enregistré précédemment

from langchain.vectorstores import FAISS

vectorstore = FAISS.load_local(
    "book_index",
    embeddings=embedding_model,
    # Ligne ci dessous à ajouter sinon erreur lors du lancement de la cellule
    allow_dangerous_deserialization=True
)

In [16]:
# Création du retriever
retriever = vectorstore.as_retriever()

In [17]:
# Initialisation du modèle de langage conversationnel Azure OpenAI

from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
                        api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=azure_deployment_name,
                        temperature=0
                        )

/tmp/ipykernel_1083/2306553836.py:5: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [18]:
# Utilisation d'un prompt personnalisé

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """
Vous êtes un assistant expert en littérature.

Utilisez les informations suivantes sur les livres pour répondre à la question de manière précise et détaillée.

Informations :
{context}

Question :
{question}

Réponse :
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [19]:
# Création de la chaine de Questions / Réponses

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    # importer le prompt personnalisé
    chain_type_kwargs={"prompt": PROMPT}
)

In [20]:
#Exemple d'utilisation 1
question = "Quels sont les thèmes principaux du livre 'Les Misérables' ?"
answer = qa_chain.run(question)
print(answer)

/tmp/ipykernel_1083/67107638.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(question)


Les thèmes principaux de "Les Misérables" de Victor Hugo incluent :

1. **Justice et Injustice** : Le roman explore les systèmes judiciaires et les inégalités sociales, illustrant comment la loi peut être à la fois oppressive et injuste, notamment à travers le personnage de Jean Valjean, qui est sévèrement puni pour un vol mineur.

2. **Rédemption et Morale** : Le parcours de Jean Valjean est central, montrant sa quête de rédemption et de transformation personnelle. Le personnage du Bishop Myriel incarne la compassion et le pardon, influençant Valjean à changer de vie.

3. **Lutte entre le Bien et le Mal** : Les personnages sont souvent confrontés à des choix moraux, et le roman examine la nature humaine à travers ces dilemmes.

4. **Pauvreté et Marginalisation** : Hugo met en lumière la misère des classes défavorisées, soulignant les conditions de vie difficiles des pauvres et des opprimés dans la société française du XIXe siècle.

5. **Amour et Sacrifice** : Les relations, notamment 

In [21]:
# Exemple d'utilisation 2
question = "Quels sont les principaux livres de Shakespeare ?"
answer = qa_chain.run(question)
print(answer)

Les principaux livres de Shakespeare incluent :

1. **"The Complete Works of William Shakespeare"** : Une anthologie complète qui rassemble ses sonnets, comédies, histoires et tragédies. Elle explore des thèmes universels tels que l'amour, l'ambition, la trahison et la complexité de la nature humaine. Les sonnets, qui ouvrent cette collection, se concentrent sur la beauté éphémère et le passage du temps.

2. **"Shakespeare's First Folio"** : Une collection de ses pièces publiée au début du 17e siècle. Elle vise à préserver son héritage littéraire malgré les défis d'impression de l'époque. Elle inclut des pièces emblématiques comme "The Tempest," "Romeo and Juliet," et "Macbeth."

3. **"Roméo et Juliette"** : Une tragédie célèbre qui raconte l'histoire d'amour tragique entre Roméo et Juliette, issus de familles rivales à Vérone. Leurs interactions poétiques et leur destin tragique sont au cœur de cette œuvre.

4. **"Périclès"** : Une tragédie qui suit les aventures du prince Périclès de

In [24]:
# Exemple d'utilisation 3
question = "Quels sont les personnages principaux de 'The Boy Trapper' et de quel sujet traite ce livre ?"
answer = qa_chain.run(question)
print(answer)

Les personnages principaux de "The Boy Trapper" de Harry Castlemon sont David Evans, son frère Dan, et Lester Brigham. Le livre traite de la détermination de David à améliorer la situation financière de sa famille en piégeant des cailles. L'histoire explore des thèmes tels que la famille, l'ambition et la rivalité. David doit naviguer à travers les défis posés par son frère Dan, qui veut une part des gains potentiels, et Lester, un nouveau venu qui menace de saboter ses efforts. Le récit se déroule dans un contexte post-guerre où David et sa mère doivent faire face aux conséquences du comportement imprudent de son père pendant la guerre, tout en essayant de subvenir aux besoins de la famille.


In [22]:
# Exemple d'utilisation 4
question = "Quels sont les thèmes principaux dans 'Les fleurs du mal' ?"
answer = qa_chain.run(question)
print(answer)

"Les Fleurs du mal" de Charles Baudelaire est une œuvre emblématique de la poésie française du 19e siècle. Les thèmes principaux de ce recueil incluent :

1. **La Beauté et la Décadence** : Baudelaire explore la dualité entre la beauté et la corruption, souvent en juxtaposant des images sublimes avec des réalités sordides.

2. **Le Spleen et l'Idéal** : Ce thème central oppose l'aspiration à l'idéal, à la beauté et à l'élévation spirituelle, au spleen, qui représente l'ennui, la mélancolie et le désespoir.

3. **L'Amour et l'Érotisme** : Les poèmes abordent les complexités de l'amour, souvent teinté de désir charnel et de souffrance.

4. **La Mort et la Mortalité** : La fascination pour la mort et l'au-delà est omniprésente, avec des réflexions sur la finitude de la vie et l'angoisse existentielle.

5. **La Ville et la Modernité** : Baudelaire décrit Paris comme un lieu de modernité, de mouvement et de transformation, tout en soulignant ses aspects aliénants.

6. **La Révolte et la Tra

In [23]:
# Exemple 5
question = "Fais moi un très bref résumé de 'Fairy Tales from the Arabian Nights' ?"
answer = qa_chain.run(question)
print(answer)

"Fairy Tales from the Arabian Nights" par E. Dixon est une collection de contes fantastiques inspirés du folklore moyen-oriental, éditée au XIXe siècle. Les histoires explorent des thèmes d'amour, de magie et d'aventure, mettant en scène des personnages comme le Prince Beder et la Reine Gulnare. Le récit débute avec un roi de Perse découvrant le passé tragique de sa reine, ouvrant la voie à des quêtes royales et des défis magiques, tout en soulignant la loyauté et la quête d'identité.


In [24]:
# Exemple d'utilisation autre que sur les livres 
question = "Quel temps fera-t-il demain ?"
answer = qa_chain.run(question)
print(answer)

Je suis désolé, mais je ne peux pas prédire la météo. Cependant, je peux vous aider avec des informations sur la littérature ou d'autres sujets. Si vous avez des questions sur les livres mentionnés ou d'autres œuvres littéraires, n'hésitez pas à me le faire savoir !


In [25]:
# Exemple d'utilisation autre que sur les livres 
question = "Aimes-tu le chocolat ?"
answer = qa_chain.run(question)
print(answer)

En tant qu'assistant virtuel, je n'ai pas de préférences personnelles, mais je peux vous dire que le chocolat est souvent apprécié pour sa saveur riche et réconfortante. Dans "The Story of a Candy Rabbit" de Laura Lee Hope, le chocolat joue un rôle dans l'univers des jouets, notamment avec le personnage du lapin en chocolat, ajoutant une touche de douceur et de magie à l'histoire. Si vous aimez les récits qui explorent l'imagination et l'amitié, ce livre pourrait vous plaire.


In [36]:
# Etape de récupération du texte complet d'un livre (si demandé par l'utilisateur) depuis le .txt sur le site
# Et répondre à des questions dessus

from langchain.document_loaders import GutenbergLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

def full_txt_question(title, question):
    """
    Pose une question sur le texte complet d'un livre.

    Args:
        title (str): Titre du livre.
        question (str): Question à poser.

    Returns:
        str: Réponse de l'agent.
    """
    # Récupération de l'ID du livre à partir du titre
    matching_books = gutenberg_data[gutenberg_data["Title"].str.lower() == title.lower()]
    if matching_books.empty:
        return f"Aucun livre intitulé '{title}' n'a été trouvé."

    book_id = matching_books.iloc[0]["EBook-No."]

    # Si le vectorstore du livre existe déja on va le charger, sinon on va charger le texte complet depuis l'url
    vectorstore_path = f"vectorstores/vectorstore_{book_id}"
    if os.path.exists(vectorstore_path):
        
        full_txt_vectorstore = FAISS.load_local(
            vectorstore_path,
            embeddings=embedding_model,
            allow_dangerous_serialization=True
        )
        print(f"Vectorstore chargé depuis {vectorstore_path}.")
    else:
    # Texte complet
        url = f"https://www.gutenberg.org/files/{book_id}/{book_id}-0.txt"
        try:
            loader = GutenbergLoader(url)
            full_txt_documents = loader.load()
            print(f"Texte complet du livre {book_id} chargé avec succès depuis {url}")
        except Exception as e:
            print(f"Erreur lors du chargement du texte {book_id}: {e}")
            return "Impossible de charger le texte complet."

# Utiliser un TextSplitter pour diviser le document en chunks sinon erreur (rate limit)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            chunk_overlap=100
        )
        split_documents = text_splitter.split_documents(full_txt_documents)

        try:
            full_txt_vectorstore = FAISS.from_documents(split_documents, embedding_model)
            # Sauvegarder le vectorstore pour une utilisation future
            full_txt_vectorstore.save_local(vectorstore_path)
            print(f"Vectorstore du livre sauvegardé dans {vectorstore_path}.")
        except Exception as e:
            print(f"Erreur lors de la création du vectorstore: {e}")
            return "Impossible de créer le vectorstore."

    retriever = full_txt_vectorstore.as_retriever()

    qa_chain_full_text = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )

    try:
            answer = qa_chain_full_text.run(question)
            return answer
    except Exception as e:
            print(f"Erreur lors de l'obtention de la réponse: {e}")
            return "Impossible de générer une réponse."

In [37]:
# Exemple d'utilisation
title = "Les Misérables"
question = "Décris la transformation de Jean Valjean tout au long du livre."

answer = full_txt_question(title, question)
print(f"Réponse : {answer}")

Texte complet du livre 135 chargé avec succès depuis https://www.gutenberg.org/files/135/135-0.txt
Vectorstore du livre sauvegardé dans vectorstores/vectorstore_135.
Réponse : La transformation de Jean Valjean dans *Les Misérables* de Victor Hugo est l'un des arcs narratifs les plus puissants et émouvants de la littérature. Voici une description détaillée de son évolution :

1. **Origine et Condamnation** : Jean Valjean commence comme un homme ordinaire, mais la pauvreté le pousse à voler un pain pour nourrir sa famille. Ce vol mineur lui vaut une condamnation sévère à cinq ans de bagne, qui se prolonge à dix-neuf ans en raison de tentatives d'évasion. Cette expérience le remplit de haine et de ressentiment envers la société.

2. **Rencontre avec l'Évêque** : Après sa libération, Valjean est accueilli par l'évêque Myriel, qui lui offre l'hospitalité. Lorsque Valjean vole l'argenterie de l'évêque, celui-ci le couvre en prétendant l'avoir donnée. Cet acte de bonté désintéressée boulevers

#### NEXT STEPS : 

##### 1. RAG Evaluation (manuelle / boucle) : 



In [38]:
test_questions = [
    {
        "question": "Quels sont les thèmes principaux du livre 'Les Misérables' ?",
        "expected_answer": "Les thèmes principaux incluent la justice sociale, la rédemption, l'amour, et la pauvreté."
    },
    {
        "question": "Qui sont les personnages principaux dans le résumé du livre 'Le Comte de Monte-Cristo' ?",
        "expected_answer": "Les personnages principaux sont Edmond Dantès, Mercédès, Fernand, Danglars, Andrea Cavalcanti, et la famille Villefort."
    },
    {
        "question": "Quels sont les thèmes principaux du livre 'Les fleurs du mal' ?",
        "expected_answer": "Les thèmes principaux sont la mélancolie, la beauté, la société, et la lutte morale."
    },
    {
        "question": "Résume moi 'Fairy Tales from the Arabian Nights' ?",
        "expected_answer": "C'est une collection de contes fantastiques inspirés du folklore moyen-oriental dont Les histoires explorent des thèmes de l'amour, de la magie et de l'aventure"
    }
]

In [43]:
def agent_evaluation(qa_chain, test_questions):
    correct_answers = 0
    total_questions = len(test_questions)

    for idx, test in enumerate(test_questions, 1):
        try:
            answer = qa_chain.run(test["question"])
            print("=" * 50)
            print(f"Question {idx}: {test['question']}")
            print(f"Réponse de l'agent: {answer}")
            print(f"Réponse attendue: {test['expected_answer']}")
            
            expected_keywords = test["expected_answer"]
            match_count = 0
            for keyword in expected_keywords:
                if keyword.lower() in answer.lower():
                    match_count += 1
            
            if match_count == len(expected_keywords):
                correct_answers += 1
                print("\n✅ Cette réponse est CORRECTE.\n")
            else:
                print(f"\n⚠️ Réponse partiellement correcte : {match_count}/{len(expected_keywords)} correspondances.\n")
        except Exception as e:
            print(f"\n❌ Erreur lors de la réponse à la question {idx}: {e}\n")
    
    accuracy = (correct_answers / total_questions) * 100
    print("=" * 50)
    print(f"Précision de l'agent: {accuracy:.2f}%")
    print("=" * 50)


In [44]:
agent_evaluation(qa_chain, test_questions)

Question 1: Quels sont les thèmes principaux du livre 'Les Misérables' ?
Réponse de l'agent: Les thèmes principaux de "Les Misérables" de Victor Hugo incluent :

1. **Justice et Injustice** : Le roman explore les systèmes judiciaires et les inégalités sociales, illustrant comment la loi peut être à la fois oppressive et injuste, notamment à travers le personnage de Jean Valjean, qui est sévèrement puni pour un vol mineur.

2. **Rédemption et Morale** : Le parcours de Jean Valjean est central, montrant sa quête de rédemption et de transformation personnelle. Le personnage du Bishop Myriel incarne la compassion et le pardon, influençant Valjean à changer de vie.

3. **Lutte entre le Bien et le Mal** : Les personnages sont souvent confrontés à des choix moraux, et le roman examine la nature humaine, la capacité de faire le bien malgré un passé trouble, et la lutte intérieure entre les forces du bien et du mal.

4. **Pauvreté et Marginalisation** : Hugo met en lumière la misère des classes

##### 1. RAG Evaluation (langSmith) : 

##### 2. Fast API

##### 3. Azure Deployment